<a href="https://colab.research.google.com/github/mohamedbakhet/Medicine-In-Take-Reminder-and-Monitor/blob/master/VGG16_1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow
! pip install h5py

In [ ]:
from keras.models import model_from_json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import numpy as np
import os
import keras
from keras import layers,models
from keras.layers import Input
from keras.models import Sequential, Model,load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten


Mounted at /content/drive


In [ ]:
#load train data set
training_data=[]
for index,value in enumerate (os.listdir('/content/drive/MyDrive/train_data')): #path to the data set on google drive
  print(value)
  lis=np.load('/content/drive/MyDrive/train_data/'+value)
  for j in lis[:1600]:
    training_data.append([np.array(j),value[:-4]])
shuffle(training_data)

#load test data set
test_data=[]
for index,value in enumerate (os.listdir('/content/drive/MyDrive/train_data')):
  print(value)
  lis=np.load('/content/drive/MyDrive/train_data/'+value)
  for j in lis[:400]:
    test_data.append([np.array(j),value[:-4]])
shuffle(test_data)

In [ ]:
#split data into train_test & encoding target

X_train = np.array([i[0] for i in training_data]).reshape(-1,300,15,1)
Y_train = np.array([i[1] for i in training_data])
train_labels_set = np.unique(Y_train,axis=0)
train_labels = LabelEncoder()
train_labels = train_labels.fit(Y_train)
Y_train = to_categorical(train_labels.transform(Y_train))

X_test = np.array([i[0] for i in test_data]).reshape(-1,300,15,1)
Y_test = np.array([i[1] for i in test_data])
test_labels_set = np.unique(Y_test,axis=0)
test_labels = LabelEncoder()
test_labels = test_labels.fit(Y_test)
Y_test = to_categorical(test_labels.transform(Y_test))

In [ ]:
#VGG16-1D
img_input = Input(shape=(300,15))
# Block 1
x = layers.Conv1D(64, 3,
                  activation='relu',
                  padding='same',
                  name='block1_conv1')(img_input)
x = layers.Conv1D(64, 3,
                  activation='relu',
                  padding='same',
                  name='block1_conv2')(x)
x = layers.MaxPooling1D(2, strides=2, name='block1_pool', padding='same')(x)

# Block 2
x = layers.Conv1D(128, 3,
                  activation='relu',
                  padding='same',
                  name='block2_conv1')(x)
x = layers.Conv1D(128, 3,
                  activation='relu',
                  padding='same',
                  name='block2_conv2')(x)
x = layers.MaxPooling1D(2, strides=2, name='block2_pool', padding='same')(x)

# Block 3
x = layers.Conv1D(256,3 ,
                  activation='relu',
                  padding='same',
                  name='block3_conv1')(x)
x = layers.Conv1D(256, 3,
                  activation='relu',
                  padding='same',
                  name='block3_conv2')(x)
x = layers.Conv1D(256, 3,
                  activation='relu',
                  padding='same',
                  name='block3_conv3')(x)
x = layers.MaxPooling1D(2, strides=2, name='block3_pool', padding='same')(x)

# Block 4
x = layers.Conv1D(512, 3,
                  activation='relu',
                  padding='same',
                  name='block4_conv1')(x)
x = layers.Conv1D(512, 3,
                  activation='relu',
                  padding='same',
                  name='block4_conv2')(x)
x = layers.Conv1D(512, 3,
                  activation='relu',
                  padding='same',
                  name='block4_conv3')(x)
x = layers.MaxPooling1D(2, strides=2, name='block4_pool', padding='same')(x)

# Block 5
x = layers.Conv1D(512, 3,
                  activation='relu',
                  padding='same',
                  name='block5_conv1')(x)
x = layers.Conv1D(512, 3,
                  activation='relu',
                  padding='same',
                  name='block5_conv2')(x)
x = layers.Conv1D(512, 3,
                  activation='relu',
                  padding='same',
                  name='block5_conv3')(x)
x = layers.MaxPooling1D(2, strides=2, name='block5_pool', padding='same')(x)

# Classification block
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(128, activation='relu', name='fc1')(x) # reduced dim for 1-d task
x = layers.Dense(128, activation='relu', name='fc2')(x)
x = layers.Dense(14, activation='softmax', name='predictions')(x)


# Create model.
model = models.Model(img_input, x, name='vgg16')

In [ ]:
#get the gpu name 
device_name = tf.test.gpu_device_name() #you can ignore this if don't have to use the gpu during the training proccess

In [ ]:
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
with tf.device(device_name): #in case that you skipped the gpu device comment these two lines and uncomment the last one and run
  hist = model.fit(X_train, Y_train, epochs=10,validation_data=(X_test,Y_test))
#hist = model.fit(X_train, Y_train, epochs=10,validation_data=(X_test,Y_test))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("VGG16-1D.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("VGG16-1D.h5")

In [ ]:
# load json and create model
json_file = open('VGG16-1D.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("VGG16-1D.h5")

In [ ]:
#to evaluate the model
opt = Adam(lr=0.001)
loaded_model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
y_predict = loaded_model.predict(X_test)
print(classification_report(Y_test.argmax(axis=1),y_predict.argmax(axis=1),target_names=test_labels_set))